In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import os 
from train import *
from tqdm import tqdm  # Import tqdm
from early_exit_resnet import EarlyExitResNet50
from calibrate import *
from torch.utils.data import DataLoader, Subset


In [2]:
# Example instantiation of the model
model = EarlyExitResNet50(num_classes=10)

device='cuda'

# Move the model to the specified device
model.load_state_dict(torch.load("models/resnet50.pth"))
model = model.to(device)


In [3]:
# torch.save(model.state_dict(),"models/resnet50.pth")

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Load the training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# Load the original test dataset
full_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(full_testset, batch_size=64, shuffle=False, num_workers=2)

# # Define the size of the validation set
# valid_size = int(0.5 * len(full_testset))  # for example, 50% of the test set

# # Generate indices: here, we shuffle the indices of the full test set and then split
# indices = torch.randperm(len(full_testset)).tolist()
# valid_indices, test_indices = indices[:valid_size], indices[valid_size:]

# # Create validation and test subsets
# validset = Subset(full_testset, valid_indices)
# testset = Subset(full_testset, test_indices)

# # Create DataLoader instances for the validation and test sets
# validloader = DataLoader(validset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [5]:

# Calculate accuracies for each exit
accuracies = evaluate_model(model, testloader, device)
for i, acc in enumerate(accuracies, 1):
    print(f'Accuracy for exit {i}: {acc * 100:.2f}%')



Accuracy for exit 1: 72.17%
Accuracy for exit 2: 74.10%
Accuracy for exit 3: 77.60%
Accuracy for exit 4: 78.07%


In [6]:
# After training your model
opt_temp_exit1, opt_temp_exit2, opt_temp_final = calibrate_model_exits(model, testloader, device)

In [ ]:
# torch.save(model.state_dict(), "resnet50-calibrated.pth")

In [6]:
def inference_with_temperature_scaling(model, inputs, temp_exit1, temp_exit2, temp_final):
    model.eval()
    softmax = torch.nn.Softmax(dim=1)
    with torch.no_grad():
        output_early_exit1, output_early_exit2, output_final = model(inputs)
        
        # Apply temperature scaling to each exit's logits
        scaled_output_early_exit1 = output_early_exit1 / temp_exit1
        scaled_output_early_exit2 = output_early_exit2 / temp_exit2
        scaled_output_final = output_final / temp_final
        
        # Calculate softmax probabilities for each exit
        probs_early_exit1 = softmax(scaled_output_early_exit1).cpu().numpy()
        probs_early_exit2 = softmax(scaled_output_early_exit2).cpu().numpy()
        probs_final = softmax(scaled_output_final).cpu().numpy()

    return probs_early_exit1, probs_early_exit2, probs_final


In [7]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

# Assuming testloader, model, and device are defined
# Initialize lists to store probabilities and true labels
probs_list_early_exit1, probs_list_early_exit2, probs_list_final = [], [], []
true_labels_list = []

softmax = torch.nn.Softmax(dim=1)
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Calculate softmax probabilities with temperature scaling for each exit
        probs_early_exit1, probs_early_exit2, probs_final = inference_with_temperature_scaling(
            model, inputs, opt_temp_exit1, opt_temp_exit2, opt_temp_final
        )
        
        # Append probabilities and true labels for later analysis
        # Assuming probs_list_early_exit1, probs_list_early_exit2, probs_list_final, and true_labels_list are defined earlier
        probs_list_early_exit1.append(probs_early_exit1)
        probs_list_early_exit2.append(probs_early_exit2)
        probs_list_final.append(probs_final)
        true_labels_list.append(labels.cpu().numpy())

# Concatenate all collected arrays
probs_early_exit1 = np.concatenate(probs_list_early_exit1, axis=0)
probs_early_exit2 = np.concatenate(probs_list_early_exit2, axis=0)
probs_final = np.concatenate(probs_list_final, axis=0)
true_labels = np.concatenate(true_labels_list, axis=0)

# Function to plot reliability diagram
def plot_reliability_diagram(y_true, y_prob, title):
    fig, ax = plt.subplots(figsize=(8, 8))
    for i in range(y_prob.shape[1]):  # Assuming y_prob.shape[1] is the number of classes
        true_class = (y_true == i)
        prob_true_class = y_prob[:, i]
        
        fraction_of_positives, mean_predicted_value = calibration_curve(true_class, prob_true_class, n_bins=10)
        
        ax.plot(mean_predicted_value, fraction_of_positives, "s-", label=f"Class {i}")
    
    ax.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    ax.set_ylabel("Fraction of positives")
    ax.set_xlabel("Mean predicted value")
    ax.set_title(title)
    ax.legend(loc="best")

# Plotting reliability diagrams for each exit
plot_reliability_diagram(true_labels, probs_early_exit1, "Reliability Diagram for Early Exit 1")
plot_reliability_diagram(true_labels, probs_early_exit2, "Reliability Diagram for Early Exit 2")
plot_reliability_diagram(true_labels, probs_final, "Reliability Diagram for Final Exit")
plt.show()


TypeError: inference_with_temperature_scaling() takes 3 positional arguments but 5 were given

In [12]:
import numpy as np
import torch

# Function to generate uncalibrated softmax probabilities
def inference_without_temperature_scaling(model, inputs):
    model.eval()
    softmax = torch.nn.Softmax(dim=1)
    with torch.no_grad():
        output_early_exit1, output_early_exit2, output_final = model(inputs)
        
        probs_early_exit1 = softmax(output_early_exit1).cpu().numpy()
        probs_early_exit2 = softmax(output_early_exit2).cpu().numpy()
        probs_final = softmax(output_final).cpu().numpy()

    return probs_early_exit1, probs_early_exit2, probs_final

# Assuming testloader, model, opt_temp_exit1, opt_temp_exit2, opt_temp_final, and device are defined
# Initialize lists to store probabilities and true labels
uncalibrated_probs_list_early_exit1, uncalibrated_probs_list_early_exit2, uncalibrated_probs_list_final = [], [], []
calibrated_probs_list_early_exit1, calibrated_probs_list_early_exit2, calibrated_probs_list_final = [], [], []
true_labels_list = []

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Generate uncalibrated probabilities
        uncal_probs_early_exit1, uncal_probs_early_exit2, uncal_probs_final = inference_without_temperature_scaling(model, inputs)
        
        # Generate calibrated probabilities
        cal_probs_early_exit1, cal_probs_early_exit2, cal_probs_final = inference_with_temperature_scaling(model, inputs, opt_temp_exit1, opt_temp_exit2, opt_temp_final)
        
        # Append uncalibrated probabilities and true labels for later analysis
        uncalibrated_probs_list_early_exit1.append(uncal_probs_early_exit1)
        uncalibrated_probs_list_early_exit2.append(uncal_probs_early_exit2)
        uncalibrated_probs_list_final.append(uncal_probs_final)
        
        # Append calibrated probabilities for later analysis
        calibrated_probs_list_early_exit1.append(cal_probs_early_exit1)
        calibrated_probs_list_early_exit2.append(cal_probs_early_exit2)
        calibrated_probs_list_final.append(cal_probs_final)
        
        # Append true labels
        true_labels_list.append(labels.cpu().numpy())


TypeError: inference_with_temperature_scaling() takes 3 positional arguments but 5 were given

In [15]:
uncalibrated_probs_early_exit1 = np.concatenate(uncalibrated_probs_list_early_exit1, axis=0)
uncalibrated_probs_early_exit2 = np.concatenate(uncalibrated_probs_list_early_exit2, axis=0)
uncalibrated_probs_final = np.concatenate(uncalibrated_probs_list_final, axis=0)

calibrated_probs_early_exit1 = np.concatenate(calibrated_probs_list_early_exit1, axis=0)
calibrated_probs_early_exit2 = np.concatenate(calibrated_probs_list_early_exit2, axis=0)
calibrated_probs_final = np.concatenate(calibrated_probs_list_final, axis=0)

true_labels = np.concatenate(true_labels_list, axis=0)


In [8]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

def plot_reliability_diagram(y_true, y_prob_uncalibrated, y_prob_calibrated, title):
    fig, ax = plt.subplots(figsize=(8, 8))
    color_sequence = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    
    for i in range(y_prob_uncalibrated.shape[1]):  # Assuming y_prob.shape[1] is the number of classes
        true_class = (y_true == i)
        prob_true_class_uncalibrated = y_prob_uncalibrated[:, i]
        prob_true_class_calibrated = y_prob_calibrated[:, i]
        
        fraction_of_positives_uncal, mean_predicted_value_uncal = calibration_curve(true_class, prob_true_class_uncalibrated, n_bins=10)
        fraction_of_positives_cal, mean_predicted_value_cal = calibration_curve(true_class, prob_true_class_calibrated, n_bins=10)
        
        ax.plot(mean_predicted_value_uncal, fraction_of_positives_uncal, "s-", label=f"Uncalibrated Class {i}", color=color_sequence[i % len(color_sequence)])
        ax.plot(mean_predicted_value_cal, fraction_of_positives_cal, "^-", label=f"Calibrated Class {i}", color=color_sequence[i % len(color_sequence)])
    
    ax.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    ax.set_ylabel("Fraction of positives")
    ax.set_xlabel("Mean predicted value")
    ax.set_title(title)
    ax.legend(loc="best")


In [9]:
# Plotting reliability diagrams for each exit
plot_reliability_diagram(true_labels, uncalibrated_probs_early_exit1, "Reliability Diagram for Early Exit 1")
# plot_reliability_diagram(true_labels, uncalibrated_probs_early_exit2, calibrated_probs_early_exit2, "Reliability Diagram for Early Exit 2")
# plot_reliability_diagram(true_labels, uncalibrated_probs_final, calibrated_probs_final, "Reliability Diagram for Final Exit")
plt.show()


NameError: name 'true_labels' is not defined

In [10]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
import numpy as np

def plot_reliability_diagram_with_cumulative_counts(y_true, y_prob, title):
    fig, ax1 = plt.subplots(figsize=(8, 8))
    ax2 = ax1.twinx()  # Instantiate a second axes that shares the same x-axis

    color_sequence = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    n_bins = 10
    bins = np.linspace(0, 1, n_bins + 1)
    
    total_counts = np.zeros(n_bins)
    
    for i in range(y_prob.shape[1]):  # Assuming y_prob.shape[1] is the number of classes
        true_class = (y_true == i)
        prob_true_class = y_prob[:, i]

        # Calibration curve
        fraction_of_positives, mean_predicted_value = calibration_curve(true_class, prob_true_class, n_bins=n_bins, strategy='uniform')
        
        # Plot calibration curve
        ax1.plot(mean_predicted_value, fraction_of_positives, "s-", label=f"Class {i}", color=color_sequence[i % len(color_sequence)])

        # Compute histogram for the predicted probabilities
        counts, _ = np.histogram(prob_true_class, bins=bins)
        total_counts += counts
        
    # Cumulative bar plot
    ax2.bar(bins[:-1], total_counts, width=bins[1]-bins[0], align='edge', color='gray', alpha=0.3, label='Counts')
    
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    ax1.set_ylabel("Fraction of positives")
    ax1.set_xlabel("Mean predicted value")
    ax1.set_title(title)
    ax1.legend(loc="best")

    ax2.set_ylabel("Counts", color='gray')
    ax2.tick_params(axis='y', labelcolor='gray')

    fig.tight_layout()  # Adjust layout to make room for the second y-axis
    plt.show()


In [11]:
plot_reliability_diagram_with_cumulative_counts()

TypeError: plot_reliability_diagram_with_cumulative_counts() missing 3 required positional arguments: 'y_true', 'y_prob', and 'title'